## Solução para a Construção do Modelo Preditivo: Banco Box

- Enunciado: O Banco Box deseja compreender melhor o perfil de seus clientes para identificar aqueles que apresentam maior risco de inadimplência, prevenindo perdas financeiras. Para isso, desenvolva um modelo de Machine Learning capaz de classificar os perfis de risco.

- **Objetivo**: Criar um modelo capaz de identificar os clientes do Banco Box que apresentam **maior risco**, com o objetivo de prevenir perdas financeiras.

### 1. Importação das Bibliotecas

In [21]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

### 2. Importar Base de Dados

In [26]:
df = pd.read_csv("data/banco_box_sintetico.csv")

In [27]:
df

,idade,renda_mensal,tempo_emprego,dividas_total,limite_cartao,historico_credito,num_cartoes_credito,num_emprestimos,atraso_pagamento,possui_imovel,possui_veiculo,cargo_atual,risco_credito
0,50,4145.83,1.4,5798,13957.29,1,1,1,0,Não,Não,Analista de TI,Médio
1,40,14080.13,0.9,39607,57549.72,1,0,2,0,Não,Não,Enfermeiro,Alto
2,59,7550.31,21.0,7865,20694.49,3,4,0,0,Não,Sim,Advogado,Médio
3,27,2390.01,5.6,1340,10593.18,17,5,3,0,Sim,Não,Analista de TI,Baixo
4,39,17135.27,20.3,45245,77949.94,1,3,2,0,Sim,Não,Contador,Alto
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,70,17133.71,10.4,36559,53726.07,7,3,0,0,Não,Não,Professor,Alto
996,82,8224.03,14.3,7642,18414.90,12,3,0,0,Sim,Não,Médico,Baixo
997,88,3286.20,19.1,1472,16055.81,12,0,1,0,Sim,Sim,Professor,Baixo
998,86,14855.40,26.6,42765,28284.90,6,3,0,0,Sim,Sim,Contador,Alto


### 3. Análise de Dados

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   idade                1000 non-null   int64  
 1   renda_mensal         951 non-null    float64
 2   tempo_emprego        1000 non-null   float64
 3   dividas_total        1000 non-null   int64  
 4   limite_cartao        1000 non-null   float64
 5   historico_credito    1000 non-null   int64  
 6   num_cartoes_credito  1000 non-null   int64  
 7   num_emprestimos      1000 non-null   int64  
 8   atraso_pagamento     1000 non-null   int64  
 9   possui_imovel        1000 non-null   object 
 10  possui_veiculo       1000 non-null   object 
 11  cargo_atual          1000 non-null   object 
 12  risco_credito        1000 non-null   object 
dtypes: float64(3), int64(6), object(4)
memory usage: 101.7+ KB


In [32]:
df.groupby("cargo_atual").size()

cargo_atual
Advogado          110
Analista TI         4
Analista de TI    112
Analista_de_TI      7
Contador          109
Designer          109
Empreendedor      116
Enfermeiro        117
Eng.                2
Engenheiro         97
Médico            107
Prof                1
Prof.               2
Professor         107
dtype: int64

In [29]:
df.groupby("risco_credito").size()

risco_credito
Alto     408
Baixo    252
Médio    340
dtype: int64

### 4. Tratamento de Dados:

In [33]:
df.loc[df["cargo_atual"] == "Analista TI", "cargo_atual"] = "Analista de TI"
df.loc[df["cargo_atual"] == "Analista_de_TI", "cargo_atual"] = "Analista de TI"
df.loc[df["cargo_atual"] == "Eng", "cargo_atual"] = "Engenheiro"

df["renda_mensal"] = df["renda_mensal"].fillna(df["renda_mensal"].mean())

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   idade                1000 non-null   int64  
 1   renda_mensal         1000 non-null   float64
 2   tempo_emprego        1000 non-null   float64
 3   dividas_total        1000 non-null   int64  
 4   limite_cartao        1000 non-null   float64
 5   historico_credito    1000 non-null   int64  
 6   num_cartoes_credito  1000 non-null   int64  
 7   num_emprestimos      1000 non-null   int64  
 8   atraso_pagamento     1000 non-null   int64  
 9   possui_imovel        1000 non-null   object 
 10  possui_veiculo       1000 non-null   object 
 11  cargo_atual          1000 non-null   object 
 12  risco_credito        1000 non-null   object 
dtypes: float64(3), int64(6), object(4)
memory usage: 101.7+ KB


### 5. Separar a Base de Dados entre Features e Classes:

In [35]:
x_data = df.iloc[:, 0:12].values
y_data = df.iloc[:, 12].values

x_data, y_data

(array([[50, 4145.83, 1.4, ..., 'Não', 'Não', 'Analista de TI'],
        [40, 14080.13, 0.9, ..., 'Não', 'Não', 'Enfermeiro'],
        [59, 7550.31, 21.0, ..., 'Não', 'Sim', 'Advogado'],
        ...,
        [88, 3286.2, 19.1, ..., 'Sim', 'Sim', 'Professor'],
        [86, 14855.4, 26.6, ..., 'Sim', 'Sim', 'Contador'],
        [83, 18825.51, 15.5, ..., 'Sim', 'Sim', 'Enfermeiro']],
       shape=(1000, 12), dtype=object),
 array(['Médio', 'Alto', 'Médio', 'Baixo', 'Alto', 'Alto', 'Alto', 'Baixo',
        'Alto', 'Médio', 'Alto', 'Alto', 'Alto', 'Alto', 'Baixo', 'Alto',
        'Baixo', 'Baixo', 'Alto', 'Médio', 'Médio', 'Médio', 'Alto',
        'Baixo', 'Alto', 'Alto', 'Médio', 'Médio', 'Alto', 'Baixo',
        'Médio', 'Médio', 'Baixo', 'Alto', 'Médio', 'Alto', 'Alto', 'Alto',
        'Médio', 'Alto', 'Médio', 'Alto', 'Médio', 'Alto', 'Baixo',
        'Médio', 'Alto', 'Médio', 'Baixo', 'Médio', 'Baixo', 'Alto',
        'Médio', 'Médio', 'Alto', 'Alto', 'Alto', 'Médio', 'Alto', 'Baixo',


#### 5.1 Segmentar colunas númericas e objetos

In [36]:
colunas_numericas = [index for index in range(x_data.shape[1]) if type(x_data[0][index]) == int or type(x_data[0][index]) == float]
colunas_objetos = [index for index in range(x_data.shape[1]) if type(x_data[0][index]) ==  str or type(x_data[0][index]) == np.str_] 

colunas_numericas, colunas_objetos

([0, 1, 2, 3, 4, 5, 6, 7, 8], [9, 10, 11])

### 6. Aplicando Pré-processamento

In [37]:
encoder_cat = ColumnTransformer(transformers=[('OneHot', OrdinalEncoder(), colunas_objetos)], remainder='passthrough')

scaler = ColumnTransformer(transformers=[('Scaler', StandardScaler(), colunas_numericas)], remainder='passthrough')

preprocessor = Pipeline(steps=[('encoder', encoder_cat),
                                ('scaler', scaler)])

x_data = preprocessor.fit_transform(x_data)

x_data

array([[-1.0, -0.9860966422066364, -1.034963584184841, ..., 1, 1, 0],
       [-1.0, -0.9860966422066364, 0.15807801141728128, ..., 0, 2, 0],
       [-1.0, 1.0140993866100703, -1.3332239830853716, ..., 4, 0, 0],
       ...,
       [1.0, 1.0140993866100703, 1.9476404048204647, ..., 0, 1, 0],
       [1.0, 1.0140993866100703, -0.7367031852843104, ..., 3, 0, 0],
       [1.0, 1.0140993866100703, 0.15807801141728128, ..., 0, 3, 0]],
      shape=(1000, 12), dtype=object)

### 7. Dividindo a Base de Dados entre treino e teste

In [38]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3)

### 8. Treinando os Modelos

#### 8.1 Regressão Logística

In [ ]:
modelLogistic = LogisticRegressionCV(solver="liblinear")
modelLogistic.fit(x_train, y_train)
y_pred = modelLogistic.predict(x_test)

In [42]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        Alto       0.87      0.93      0.90       121
       Baixo       0.88      0.90      0.89        67
       Médio       0.84      0.78      0.81       112

    accuracy                           0.86       300
   macro avg       0.87      0.87      0.86       300
weighted avg       0.86      0.86      0.86       300



#### 8.2 SVM (Support Vector Machines)

In [51]:
modelSVC = SVC()
modelSVC.fit(x_train, y_train)
y_pred = modelSVC.predict(x_test)

In [52]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        Alto       0.87      0.67      0.76       121
       Baixo       0.65      0.76      0.70        67
       Médio       0.60      0.69      0.64       112

    accuracy                           0.70       300
   macro avg       0.71      0.71      0.70       300
weighted avg       0.72      0.70      0.70       300



#### 8.3 Random Forest Classifier

In [45]:
modelRFC = RandomForestClassifier()
modelRFC.fit(x_train, y_train)
y_pred = modelRFC.predict(x_test)

In [46]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        Alto       0.84      0.90      0.87       121
       Baixo       0.86      0.90      0.88        67
       Médio       0.81      0.73      0.77       112

    accuracy                           0.84       300
   macro avg       0.84      0.84      0.84       300
weighted avg       0.84      0.84      0.83       300



#### 8.4 Gradient Boosting Classifier

In [47]:
modelGBC = GradientBoostingClassifier()
modelGBC.fit(x_train, y_train)
y_pred = modelGBC.predict(x_test)

In [48]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        Alto       0.95      0.88      0.91       121
       Baixo       0.88      0.97      0.92        67
       Médio       0.86      0.87      0.86       112

    accuracy                           0.90       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.90      0.90      0.90       300



#### 8.5 KNeighborsClassifier

In [49]:
modelKNN = KNeighborsClassifier()
modelKNN.fit(x_train, y_train)
y_pred = modelKNN.predict(x_test)

In [50]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        Alto       0.57      0.71      0.63       121
       Baixo       0.47      0.52      0.50        67
       Médio       0.47      0.32      0.38       112

    accuracy                           0.52       300
   macro avg       0.51      0.52      0.50       300
weighted avg       0.51      0.52      0.51       300



### 9. Importando o Melhor Modelo

In [ ]:
import joblib 

joblib.dump(modelGBC, "modelRFC.pkl")
joblib.dump(preprocessor, "preprocessor.pkl")